In [1]:
from typing import Literal
from tqdm import tqdm
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_experimental.plan_and_execute import (
    load_chat_planner, load_agent_executor, PlanAndExecute
)

import pandas as pd

In [2]:
ReasoningStrategies = Literal["zero-shot-react", "plan-and-solve"]

def load_agent(
    tool_names: list[str],
    strategy: ReasoningStrategies = "zero-shot-react"
) -> LLMChain:
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    tools = load_tools(
        tool_names=tool_names,
        llm=llm
    )
    
    if strategy == "plan-and-solve":
        planner = load_chat_planner(llm)
        executor = load_agent_executor(llm, tools, verbose=True)
        return PlanAndExecute(planner=planner, executor=executor, verbose=True)

    return initialize_agent(
        tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

In [3]:
# 예시 도구 이름 리스트
tool_names = ["wikipedia"]

# 에이전트 로드
agent_chain = load_agent(tool_names, strategy="plan-and-solve")

protein_name = "Cyclin-dependent kinase 7"
query = f"Please provide a brief description of {protein_name}."

# 에이전트 실행
result = agent_chain.invoke(query)



> Entering new PlanAndExecute chain...
steps=[Step(value='Define Cyclin-dependent kinase 7 (CDK7) and its role in the cell cycle.'), Step(value='Explain the function of CDK7 in relation to cyclins and its involvement in transcription regulation.'), Step(value='Mention any known implications of CDK7 in diseases, particularly cancer.'), Step(value="Summarize the key points in a concise manner.\nGiven the above steps taken, please respond to the user's original question. \n")]

> Entering new AgentExecutor chain...
Thought: I need to provide a definition of Cyclin-dependent kinase 7 (CDK7) and explain its role in the cell cycle. To do this, I will look up information on CDK7 to ensure accuracy and comprehensiveness.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Cyclin-dependent kinase 7"
}
```

Observation: Page: Cyclin-dependent kinase 4
Summary: Cyclin-dependent kinase 4 also known as cell division protein kinase 4 is an enzyme that in humans is encoded by the CDK4 gene. 

In [6]:
print(result['output'])

Cyclin-dependent kinase 7 (CDK7) is an enzyme that regulates the cell cycle and is part of a trimeric complex with cyclin H and MAT1. It acts as a Cdk-activating kinase (CAK) and is crucial for the phosphorylation of RNA polymerase II, facilitating transcription initiation and elongation. CDK7 also plays a role in DNA repair through its involvement in the TFIIH complex. Its dysregulation is implicated in various diseases, including cancer.
